In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

----

Ein Perceptron mit einem Input

In [10]:
p = nn.Linear(1,1)
p.weight.data = torch.tensor([3.0])
p.bias.data = torch.tensor([2.0])
x = torch.FloatTensor([[[2.0]]])
#x = torch.tensor([2.0])
print(list(p.parameters()))
p(x)                          


[Parameter containing:
tensor([3.], requires_grad=True), Parameter containing:
tensor([2.], requires_grad=True)]


tensor([[8.]], grad_fn=<AddBackward0>)

----

Ein Perceptron mit zwei Inputs

In [116]:
p = nn.Linear(2,1)
p.weight.data = torch.tensor([3.0, 2.0])
p.bias.data = torch.tensor([1.0])
print(list(p.parameters()))
x = torch.FloatTensor([[[2.0,6.0]]])
p(x)
 

[Parameter containing:
tensor([3., 2.], requires_grad=True), Parameter containing:
tensor([1.], requires_grad=True)]


tensor([[19.]], grad_fn=<AddBackward0>)

-----

## Der Gradient

#### Beispiel 1:

Bestimme den Wert der Funktion Funktion $f(x_1,x_2,x_3) = (x_1^2 + 3x_2) \cdot 2x_3$ am Punkt $x = (2,-1,3)$.
Gehe von $x$ aus um $0.01$ Längeneinheiten in Richtung des negativen Gradienten zum Punkt $x'$ und werte dort die Funktion erneut aus.

<img src="./img/nn_19.png" width="800"/>    


Der gesuchte Gradient ist $(24, 18, 2)$

In [51]:
x = torch.tensor([2.0,-1.0,3.0],requires_grad=True)
x1,x2,x3 = x[0],x[1],x[2]
a = x1 * x1
b = 3 * x2
c = 2 * x3
d = a + b
e = c * d

a.retain_grad()
b.retain_grad()
c.retain_grad()
d.retain_grad()
e.retain_grad()
x1.retain_grad()

e.backward()

print(e)

print("a.grad",a.grad)
print(b.grad.numpy())
print(c.grad)
print(d.grad)
print(e.grad)
print(x1.grad)
print(x1.grad/a.grad)

tensor(6., grad_fn=<MulBackward0>)
a.grad tensor(6.)
6.0
tensor(1.)
tensor(6.)
tensor(1.)
tensor(24.)
tensor(4.)


In [22]:
x = torch.tensor([2.0,-1.0,3.0],requires_grad=True)
 
y = (x[0]*x[0] + 3*x[1])*2*x[2]
 
y.backward()

x_given = x.detach().numpy()
print("gegebener Punkt: ",x_given)
print("Auswertung der Funktion:", y.detach().numpy())

grad = x.grad.numpy()
print("Gradient: ",grad)

gradLength = np.linalg.norm(grad)   
print("Länge des Gradienten:", gradLength )

descent = -grad/gradLength
print("normierter negativer Gradient",  descent)

learning_rate = 0.01
x_new = x.detach().numpy() + learning_rate * descent
print("neuer Punkt: ",x_new)

x = Variable(torch.FloatTensor(x_new),requires_grad=True)
y = (x[0]*x[0] + 3*x[1])*2*x[2]
 
print("Auswertung der Funktion:", y.detach().numpy())

gegebener Punkt:  [ 2. -1.  3.]
Auswertung der Funktion: 6.0
Gradient:  [24. 18.  2.]
Länge des Gradienten: 30.066593
normierter negativer Gradient [-0.79822814 -0.5986711  -0.06651901]
neuer Punkt:  [ 1.9920177 -1.0059867  2.9993348]
Auswertung der Funktion: 5.6997833


In [27]:
x = torch.tensor([2.0,-1.0,3.0])
print(x)
 
print(x1.numpy())


tensor([ 2., -1.,  3.])
[ 2. -1.  3.]


In [38]:
# Einen Tensor erstellen, für den man keinen Gradienten braucht, z.B. für die Daten
x = torch.tensor([2.0,-1.0,3.0])   # requires_grad=False ist default
xn = x.numpy()
print(x, xn)


tensor([ 2., -1.,  3.]) [ 2. -1.  3.]


In [39]:
# Einen Tensor erstellen, für den man einen Gradienten benötigt.
x = torch.tensor([2.0,-1.0,3.0],requires_grad=True)   # requires_grad=False ist default
xd = x.detach()       # ohne Gradient
xn = xd.numpy()
print(x, xd, xn)

tensor([ 2., -1.,  3.], requires_grad=True) tensor([ 2., -1.,  3.]) [ 2. -1.  3.]


In [19]:
import numpy as np

x = Variable(torch.FloatTensor([2.0,-1.0,3.0]),requires_grad=True)
y = (x[0]*x[0] + 3*x[1])*2*x[2]
 
y.backward()

x_given = x.detach().numpy()
print("gegebener Punkt: ",x_given)
print("Auswertung der Funktion:", y.detach().numpy())

grad = x.grad.numpy()
print("Gradient: ",grad)

gradLength = np.linalg.norm(grad)   
print("Länge des Gradienten:", gradLength )

descent = -grad/gradLength
print("normierter negativer Gradient",  descent)

learning_rate = 0.01
x_new = x.detach().numpy() + learning_rate * descent
print("neuer Punkt: ",x_new)

x = Variable(torch.FloatTensor(x_new),requires_grad=True)
y = (x[0]*x[0] + 3*x[1])*2*x[2]
 
print("Auswertung der Funktion:", y.detach().numpy())

gegebener Punkt:  [ 2. -1.  3.]
Auswertung der Funktion: 6.0
Gradient:  [24. 18.  2.]
Länge des Gradienten: 30.066593
normierter negativer Gradient [-0.79822814 -0.5986711  -0.06651901]
neuer Punkt:  [ 1.9920177 -1.0059867  2.9993348]
Auswertung der Funktion: 5.6997833


#### Beispiel 2:
Bestimme den Wert der Funktion $f(x_1,x_2,x_3) = (x_1\cdot x_2 + x_1^2) \cdot (x_3 - 5)$ am Punkt $x = (1,-2,2)$.
Gehe von $x$ aus um $0.01$ Längeneinheiten in Richtung des negativen Gradienten zum Punkt $x'$ und werte dort die Funktion erneut aus.
(Mit etwas Übung kann man bei einfachen Funktionen die partiellen Ableitungen direkt hinschreiben). 

<img src="./img/nn_20.png" width="800"/>  

In [53]:
import numpy as np

x = Variable(torch.FloatTensor([1.0,-2.0,2.0]),requires_grad=True)
y = (x[0]*x[1] + x[0] * x[0])*(x[2]-5)
 
y.backward()

x_given = x.detach().numpy()
print("gegebener Punkt: ",x_given)
print("Auswertung der Funktion:", y.detach().numpy())

grad = x.grad.numpy()
print("Gradient: ",grad)

gradLength = np.linalg.norm(grad)   
print("Länge des Gradienten:", gradLength )

descent = -grad/gradLength
print("normierter negativer Gradient",  descent)

learning_rate = 0.01
x_new = x.detach().numpy() + learning_rate * descent
print("neuer Punkt: ",new_x)

x = Variable(torch.FloatTensor(x_new),requires_grad=True)
y = (x[0]*x[1] + x[0] * x[0])*(x[2]-5)
 
print("Auswertung der Funktion:", y.detach().numpy())

gegebener Punkt:  [ 1. -2.  2.]
Auswertung der Funktion: 3.0
Gradient:  [ 0. -3. -1.]
Länge des Gradienten: 3.1622777
normierter negativer Gradient [-0.          0.94868326  0.31622776]
neuer Punkt:  [ 1.9920177 -1.0059867  2.9993348]
Auswertung der Funktion: 2.9684072


---

Für die Daten soll das richtige Gewicht gefunden werden:  

In [49]:
import torch.optim as optim

data = [(1,3), (2,6), (3,9), (4,12), (5,15), (6,18)]

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(1,1)
    def forward(self, x):
        x = self.fc1(x)
        return x

def criterion(out, label):
    return (label - out)**2

net = Net()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.5)

for epoch in range(100):
    for i, data2 in enumerate(data):
        X, Y = iter(data2)
        X, Y = Variable(torch.FloatTensor([X]), requires_grad=True),Variable(torch.FloatTensor([Y]), requires_grad=False)
        optimizer.zero_grad()
        outputs = net(X)
        loss = criterion(outputs, Y)
        loss.backward()
        optimizer.step()
        if (i % 10 == 0):
            print("Epoch {} - loss: {}".format(epoch, loss.data[0])) 
            
print(list(net.parameters()))



Epoch 0 - loss: 7.992253303527832
Epoch 1 - loss: 2.235430955886841
Epoch 2 - loss: 0.385786771774292
Epoch 3 - loss: 0.7099998593330383
Epoch 4 - loss: 0.5065121054649353
Epoch 5 - loss: 0.4985172748565674
Epoch 6 - loss: 0.4343971610069275
Epoch 7 - loss: 0.3958275020122528
Epoch 8 - loss: 0.3547412157058716
Epoch 9 - loss: 0.3198792636394501
Epoch 10 - loss: 0.28778746724128723
Epoch 11 - loss: 0.2591332495212555
Epoch 12 - loss: 0.23325933516025543
Epoch 13 - loss: 0.20999310910701752
Epoch 14 - loss: 0.18903934955596924
Epoch 15 - loss: 0.17017938196659088
Epoch 16 - loss: 0.1532001942396164
Epoch 17 - loss: 0.13791537284851074
Epoch 18 - loss: 0.12415505945682526
Epoch 19 - loss: 0.11176783591508865
Epoch 20 - loss: 0.10061671584844589
Epoch 21 - loss: 0.09057796746492386
Epoch 22 - loss: 0.0815407782793045
Epoch 23 - loss: 0.07340516149997711
Epoch 24 - loss: 0.0660814642906189
Epoch 25 - loss: 0.05948852002620697
Epoch 26 - loss: 0.053553249686956406
Epoch 27 - loss: 0.04821000

In [14]:
data = [(1,3), (2,6), (3,9), (4,12), (5,15), (6,18)]
for i, data2 in enumerate(data):
    X, Y = data2
    print(X,Y)

1 3
2 6
3 9
4 12
5 15
6 18


In [50]:
x = Variable(torch.FloatTensor([-1.0,2.0]),requires_grad=True)
y = 2*x[0] + 3*x[1]
x.grad


tensor(4., grad_fn=<AddBackward0>)


tensor([2., 3.])

In [60]:
x = torch.tensor(([1.0]),requires_grad=True)
y = x**2
z = 2*y
w= z**3

# This is the subpath
# Do not use detach()
p = z
q = torch.tensor(([2.0]), requires_grad=True)
pq = p*q
#pq.backward(retain_graph=True)

w.backward()
print(y,z,w,p,pq)
print(x.grad)

tensor([1.], grad_fn=<PowBackward0>) tensor([2.], grad_fn=<MulBackward0>) tensor([8.], grad_fn=<PowBackward0>) tensor([2.], grad_fn=<MulBackward0>) tensor([4.], grad_fn=<MulBackward0>)
tensor([48.])
